# DifferentialEquations.jl

[DifferentialEquations.jl](https://docs.sciml.ai/DiffEqDocs/dev/index.html) is by far [the best](https://docs.sciml.ai/SciMLBenchmarksOutput/stable/MultiLanguage/ode_wrapper_packages/) free and open source differential equations solver (not just for Julia, for any language): it is orders of magnitude faster and has orders of mangitude more features than anything else out there. It can solve standard ODEs, Delay-DEs, stochastic DEs, PDEs, ADEs, event handling, 1000s of solvers for all tese DEs and many other features. It is the central part of a whole organization focused on scientific machine learning (SciML) and the basis for [ModelingToolkit.jl](https://docs.sciml.ai/ModelingToolkit/dev/), a library for building simulations from symbolic definitions.

Here we will focus on Ordinary Differential Equations (ODEs) solving and use the module `OrdinaryDiffEq` only.

## Defining and solving some ODEs

The way DifferentialEquations.jl works is quite straightforward:

1. Make your set of ODEs a Julia function `f`
2. Put `f`, an initial state and a parameter container into an `ODEProblem`.
2. Choose the solvers and the arguments of the solvers you will use (e.g. tolerances, etc.)
3. Give the `ODEProblem` as well as the auxiliary arguments to the function `solve`!

Let's see it in practice by solving the Thomas cyclical system

$$
\begin{aligned}
\dot{x} &= \sin(y) - bx\\
\dot{y} &= \sin(z) - by\\
\dot{z} &= \sin(x) - bz
\end{aligned}
$$

First step is to make this a Julia function `f`. There are different possibilities on how to define this `f`, but here we will focus on the simplest case, called "out of place". The function function must be in the form `f(u, p, t) -> udot`, which means that given a state `u`, parameters `p` and current time `t` it returns a state vector `udot` containing the rates of change $d\mathbf{x}/dt$. 
For both `u, udot` it is recommended to use **static vectors** `SVector` for ODEs with small number of variables.

In [ ]:
import Pkg
Pkg.activate(joinpath(@__DIR__, ".."))

In [ ]:
using OrdinaryDiffEqTsit5, CairoMakie
using StaticArrays: SVector

function thomas_rule(u, p, t)
    x,y,z = u
    b = p[1]
    xdot = sin(y) - b*x
    ydot = sin(z) - b*y
    zdot = sin(x) - b*z
    return SVector(xdot, ydot, zdot)
end

Then set the initial state and parameter container

In [ ]:
u₀ = SVector(1.0, 1.0, 2.0)
p₀ = [0.2]

then put everything into the `ODEProblem` structure:

In [ ]:
# third argument is the timespan to solve in
timespan = (0.0, 1000.0)
prob = ODEProblem(thomas_rule, u₀, timespan, p₀)

Alright, so now we choose the solver algorithm to use. We will discuss this choice in more detail later, but for now we use the default choice of DifferentialEquations.jl.

In [11]:
alg = Tsit5();

Now let's solve the Lorenz system using the default settings of DifferentialEquations.jl (without specifying anything else).

In [ ]:
sol = solve(prob; alg) # provide solver algorithm as keyword `alg`

Okay, so what is the returned result? We didn't specify when to save or anything...

The solver we chose, `Tsit5`, is an _adaptive step solver_. The system is evolved with an adaptive step size, so that the step error tolerance stays below a pre-defined level. The system is evolved until we reach the end of the time span. A state is recorded at every step the solver algorithm takes naturally. E.g. the solution at the third step is

In [ ]:
sol.t

The solution is guaranteed to start and stop at the limits of the time span (by default).

We can obtain the state at the 3rd time point from the `u` field:

In [ ]:
(sol.t[3], sol.u[3])

Because of the mechanics of the solver, `sol` object allows arbitrary interpolation in time by saving some extra derivative-related numbers. This means that we can use `sol` as a function of time, `sol(t)`, to obtain the solution at time `t`

In [ ]:
sol(5.6)

We can iterate over some time to create a (multivariate) timeseries

In [ ]:
t = timespan[1]:0.1:timespan[end]
X, Y, Z = zero(t), zero(t), zero(t)
for (i, τ) in enumerate(t)
    X[i], Y[i], Z[i] = sol(τ)
end
lines(X, Y, Z; axis = (type = Axis3, azimuth = 0.75π))

Notice that it is possible to skip this advanced feature of interpolation (and thus also skip collecting the extra interpolation-related numbers) and only save at some pre-defined time points by passing the extra keyword argument `saveat = points_I_want` to the `solve` call.

In [ ]:
t = timespan[1]:0.1:timespan[end]
sol = solve(prob; alg, saveat = t)
sol.t

## Using callbacks for triggering events during time evolution

DifferentialEquations.jl implements a simple interface so that "events" (arbitrary things) can be triggered to after your solution as it is progressing in time. This is done with the [`Callback` interface](https://docs.sciml.ai/DiffEqDocs/dev/features/callback_functions/). For example, a `ContinuousCallback` represents an event that "triggers" when a continuous univariate function of the (state, time) reaches the value 0. Similarly, a `DiscreteCallback` triggers when a Boolean function of the (state, time) evaluates to `true`.

Let's modify the Thomas cyclical system so that a condition triggers each time the third variable becomes zero:


In [ ]:
# The third argument of this function is mandatory but too advanced for this lecture
condition(u, t, integrator) = u[3]

We then define a function that modifies the integrator object. the integrator object is too complicated to explain within this lecture; but we mainly care about the fields `.u` and `.p` that access current state and parameter values. 

In [ ]:
function affect!(integrator)
    integrator.u = 1.5integrator.u
end

The trigger `condition` and how it will `affect!` the integration are wrapped in continuous callback, which is given to the `solve` call.

In [20]:
cb = ContinuousCallback(condition, affect!);

In [ ]:
sol = solve(prob; alg, saveat = t, callback = cb)
# plot
lines(sol.u; axis = (type = Axis3,))

## Choosing a solver

So far we have not discussed _how_ the ODE is solved. 
A solver algorithm is chosen when a solution is requested.
So far we have been using `Tsit5`, which is also the overall default that DifferentialEquations.jl uses. But, DifferentialEquations.jl has an impressive list of [100s of solvers one can choose from](https://docs.sciml.ai/DiffEqDocs/dev/solvers/ode_solve/).

Choosing the solver is a problem-dependent operation. We'll cover here two typical examples.




### Higher accuracy, higher order

The solver `Tsit5` is most performant when medium-high error tolerances are requested. When we require very small errors, choosing a different solver can be more accurate. This can be especially impactful for chaotic dynamical systems. Let's first explicitly ask for a given accuracy when solving the ODE by passing the keywords `abstol, reltol` (for absolute and relative tolerance respectively):


In [28]:
using OrdinaryDiffEqVerner

In [ ]:
sol = solve(prob; alg, saveat = t, abstol = 1e-12, reltol = 1e-12)
sol[end]

Let's now benchmark how much time it takes to solve the problem with the default `Tsit5` solver:

In [ ]:
using BenchmarkTools
@btime solve(prob; alg, saveat = t, abstol = 1e-12, reltol = 1e-12);

Let's see how the higher-order solver `Vern9`, which is better suited for high accuracy, performs:

In [ ]:
using BenchmarkTools
@btime solve(prob; alg = Vern9(), saveat = t, abstol = 1e-12, reltol = 1e-12);

## Stiff problems

A "stiff" ODE problem is one that can be numerically unstable unless the step size (or equivalently, the step error tolerances) are extremely small. There are several situations where a problem may be come "stiff":

- The derivative values can get very large for some state values.
- There is a large _timescale separation_ between the dynamics of the variables
- There is a large _timescale separation_ between the dynamics of different state space regions

One must be aware whether this is possible for their system and choose a solver that is better suited to tackle stiff problems. If not, a solution may diverge and the ODE integrator will throw an error or a warning.

  Many of the problems in DifferentialEquations.jl are suitable for dealing with stiff problems, such as `Rodas5P()`. Consult the solver documentation for the possibilities.

# Exercises


## Bouncing ball

Using the callback functionality of DifferentialEquations.jl, implement the bouncing ball physical system:

$$
\begin{aligned}
\dot{x} &= v \\
\dot{v} &= -g - \gamma v 
\end{aligned}
$$
and at $x=0$ there is a table that the ball bounces from. $g$ is the gravity constant and $\gamma$ the air friction (use e.g. $g=10, \gamma = 0.99, x = 1, v = 0$). Implement this problem for elastic collisions (elastic collisions preserve velocity measure), and plot the time evolution of $x, v$ versus time.

*Hint: whenever the ball reaches the level $x=0$, its velocity should be reversed.*

## The N-body problem

The n-body problem involves solving a second-order differential equation. Normally this is solved by writing a coupled first-order differential equation, with 

$$\vec{u} = \begin{bmatrix} \vec{r} \\ \vec{v} \end{bmatrix}, \qquad \frac{d \vec{u}}{dt} = \begin{bmatrix} \vec{v} \\ \vec{a} \end{bmatrix} $$

where $\vec{r}$ is the positional vector, $\vec{v}$ is the velocity vector, $\vec{a}$ is the acceleration vector given by

$$\vec{a}_i = \sum_{i=1, i \neq j}^N -G\frac{m_j}{r_{ij}^2}\frac{\vec{r_{ij}}}{r_{ij}}.$$

With $\vec{r}_{ij} = \vec{r}_i - \vec{r}_j$, and $r_{ij} = |\vec{r}_{ij}|$.

Write a code for solving the N-body problem. Try with 2, 3, 4, or more particles. Experiment with different solvers. 

### Bonus exercise

 `DifferentialEquations.jl` contains solvers specialized for second-order ODEs, which may in some cases provide better performance and accuracy on specific problems. Write your N-body solver to be in the form of a 2nd order differential equation, and solve it using one of the [second order solvers](https://docs.sciml.ai/DiffEqDocs/stable/types/dynamical_types/). Benchmark your code with the first-order version. Do you see a difference in performance and/or accuracy?